In [65]:
import geemap
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import pdb
from IPython.display import display
import ee
import os

In [66]:
#ee.Authenticate()
#geemap.update_package()

ee.Initialize()

Map = geemap.Map(center=[31.539096,-81.422318], zoom=10)

# ##Adding every plot coordinate
# allplots_ga = 'C:/Users/arj26323/Documents/Data/Biomass datasets/Sapelo/GA_allplots_NEW.csv'
# allplots_va= pd.read_csv('C:/Users/arj26323/Documents/Data/Biomass datasets/Virginia/Coordinates and biomass/va_allpoints.csv')
# allplots_va = allplots_va[allplots_va['liveMass'].notna()]
# allplots_va = allplots_va[allplots_va['Latitude'].notna()]
# allplots_va = allplots_va.drop(columns = ['deadMass', 'collectDate', 'totalMass', 'Ell', 'MSL', 'latitude', 'longitude'])
# df_temp=allplots_va.copy()
# df_temp = df_temp.groupby(['Year','locationID','Transect','Latitude','Longitude'],as_index = False).aggregate(
#     {
#         'liveMass':[np.mean, np.size]
#     }
# )
# df_temp.columns = [
#     'Year','locationID','Transect','Latitude','Longitude','liveMass','N'
# ]
# allplots_va = df_temp
# s_patens = pd.read_csv('C:/Users/arj26323/Documents/Data/Biomass datasets/Massachusetts/LTE-MP-LPP-biomass.csv')
# s_alterniflora = pd.read_csv('C:/Users/arj26323/Documents/Data/Biomass datasets/Massachusetts/LTE-MP-LPA-biomass.csv')
# s_patens['Latitude'] = 42.730953216553
# s_patens['Longitude'] = -70.842918395996
# s_alterniflora['Latitude'] = 42.731742858887
# s_alterniflora['Longitude'] = -70.842468261719
# s_patens.rename(columns={'LIVE biomass':'liveMass'}, inplace=True)
# s_alterniflora.rename(columns={'MEAN BIOMASS':'liveMass'}, inplace=True)
# df = pd.concat([s_patens, s_alterniflora])
# df['Date'] = pd.to_datetime(df[['YEAR', 'MONTH', 'DAY']])
# dfx =  df.groupby(['SITE', 'Date'], as_index = False).aggregate(
#     {
#         'Longitude':[np.mean], 'Latitude':[np.mean], 'liveMass':[np.mean, np.size], 
#         'MONTH':'first', 'YEAR':'first'
#     }
# ) 
# dfx.columns = [
#     'Site','Date','Longitude','Latitude','liveMass', 'Sample size','Month', 'Year'
# ]

# dfx.loc[dfx['Site'] == 'LTE-MP-LPP', 'Latitude'] = 42.730953216553
# dfx.loc[dfx['Site'] == 'LTE-MP-LPP', 'Longitude'] = -70.842918395996
# dfx.loc[dfx['Site'] == 'LTE-MP-LPA', 'Latitude'] = 42.731742858887
# dfx.loc[dfx['Site'] == 'LTE-MP-LPA', 'Longitude'] = -70.842468261719

# fc_ma = geemap.pandas_to_ee(dfx, latitude = 'Latitude', longitude = 'Longitude')
# fc_va = geemap.pandas_to_ee(allplots_va, latitude = "Latitude", longitude = "Longitude")
# fc_ga = geemap.csv_to_ee(allplots_ga, latitude = "Latitude", longitude = "Longitude")

# Map.addLayer(fc_ma, {}, "MA")
# Map.addLayer(fc_va, {}, "VA")
# Map.addLayer(fc_ga, {}, "GA")

# # Map

In [191]:
yr_start = 2021
yr_end = 2021

ls_ma = ee.ImageCollection('LANDSAT/LC08/C02/T1_L2').filterBounds(fc_ma) \
    .filter(ee.Filter.calendarRange(yr_start, yr_end,'year'))

ls_va = ee.ImageCollection('LANDSAT/LC08/C02/T1_L2').filterBounds(fc_va) \
    .filter(ee.Filter.calendarRange(yr_start, yr_end,'year'))

ls_ga = ee.ImageCollection('LANDSAT/LC08/C02/T1_L2').filterBounds(fc_ga) \
    .filter(ee.Filter.calendarRange(yr_start, yr_end,'year'))

In [192]:
# Apply scaling factors, cloud mask, and FLATS
def addFLATS(image):
    flats = ee.Image(0).expression(
        '1/(1+2.718281828459045**-(-1.57 + 20*(RED-SWIR)/(RED+SWIR) + 68.6*(GREEN-RED)/(GREEN+RED)))', {
            'SWIR': image.select('SR_B6'),
            'RED': image.select('SR_B4'),
            'GREEN': image.select('SR_B3')
        })
    
    return image.addBands(flats.rename('flats'))

def maskFLATS(image):
    mask1 = image.select('flats').lte(0.1) #less than or equal to 0.1 - change?
    return image.updateMask(mask1)

def addpheno(image):
    pheno = ee.Image(0).expression(
        '(GREEN-RED)/(GREEN+RED)', {
            'RED': image.select('SR_B4'),
            'GREEN': image.select('SR_B3')
        })
    
    return image.addBands(pheno.rename('pheno'))

def addndvi(image):
    ndvi = ee.Image(0).expression(
        '(NIR-RED)/(NIR+RED)', {
            'RED': image.select('SR_B4'),
            'NIR': image.select('SR_B5')
        })
    
    return image.addBands(ndvi.rename('ndvi'))

def addnirv(image):
    nirv = ee.Image(0).expression(
        'NIR*(NIR-RED)/(NIR+RED)', {
            'RED': image.select('SR_B4'),
            'NIR': image.select('SR_B5')
        })
    
    return image.addBands(nirv.rename('nirv'))

def addevi(image):
    evi = ee.Image(0).expression(
        '(NIR-RED)/(NIR+6*RED-7.5*BLUE+1)', {
            'RED': image.select('SR_B4'),
            'NIR': image.select('SR_B5'),
            'BLUE': image.select('SR_B2'),

        })
    
    return image.addBands(evi.rename('evi'))

def maskL8sr(image):
    qaMask = image.select('QA_PIXEL').bitwiseAnd(int('11111', 2)).eq(0)
    saturationMask = image.select('QA_RADSAT').eq(0)
    # Apply the scaling factors to the appropriate bands.
    opticalBands = image.select('SR_B.').multiply(0.0000275).add(-0.2)
    thermalBands = image.select('ST_B.*').multiply(0.00341802).add(149.0)
    # Replace the original bands with the scaled ones and apply the masks.
    return image.addBands(opticalBands, None, True) \
    .addBands(thermalBands, None, True) \
    .updateMask(qaMask) \
    .updateMask(saturationMask)

scaled_ma = ls_ma.map(maskL8sr).map(addFLATS).map(maskFLATS).map(addpheno).map(addndvi).map(addevi).map(addnirv)
scaled_va = ls_va.map(maskL8sr).map(addFLATS).map(maskFLATS).map(addpheno).map(addndvi).map(addevi).map(addnirv)
scaled_ga = ls_ga.map(maskL8sr).map(addFLATS).map(maskFLATS).map(addpheno).map(addndvi).map(addevi).map(addnirv)

In [193]:
#https://developers.google.com/earth-engine/apidocs/ee-imagecollection-getregion

#alternatively, ee.image sample? https://developers.google.com/earth-engine/api_docs#ee.image.sample

In [194]:
#LTER domains:
gce_clip = 'F:/Wetlands shapefiles/GA_shapefile_wetlands/GA_simple.shp' ##Simplified GCE domain - reduced number of vertices
vcr_clip = 'F:/Wetlands shapefiles/VA_shapefile_wetlands/VA_simple.shp' ##Simplified VCR domain - reduced number of vertices
pie_clip = 'F:/Wetlands shapefiles/MA_shapefile_wetlands/MA_simple.shp' ##Simplified PIE domain - reduced number of vertices
wet_ga = geemap.shp_to_ee(gce_clip)
wet_va = geemap.shp_to_ee(vcr_clip)
wet_ma = geemap.shp_to_ee(pie_clip)
joinedFeatures = wet_ga.union()

# clip_ga = scaled_ga.map(lambda image: image.clip(wet_ga))
# clip_0 = ee.Image(clip_ga.toList(clip_ga.size()).get(1)) ##Second image in collection

# Map.addLayer(clip_0, l8_param, 'Landsat 2020')
Map.addLayer(ee.Image(scaled_ga.toList(scaled_ga.size()).get(0)), l8_param, '2020 Landsat 8 collection')
Map.addLayer(joinedFeatures, {}, 'GCE domain')
Map

Map(bottom=51233.0, center=[37.183552769849015, -75.24331873511612], controls=(WidgetControl(options=['positio…

In [196]:
#Time series of pheno band

#Add NIRV index! NIR*ndvi

out_dir = os.path.expanduser('~/Downloads')

out_ga = os.path.join(out_dir, 'nirv_ga21.csv')
out_va = os.path.join(out_dir, 'nirv_va21.csv')
out_ma = os.path.join(out_dir, 'nirv_ma21.csv')

collection_ga = scaled_ga \
                  .select("nirv")

collection_va = scaled_va \
                  .select("nirv")

collection_ma = scaled_ma \
                  .select("nirv")

# geemap.zonal_statistics(in_value_raster = collection_ga, in_zone_vector = wet_ga, \
#                 out_file_path = out_ga, statistics_type='MEAN', \
#                 scale = 30)

# geemap.zonal_statistics(in_value_raster = collection_va, in_zone_vector = wet_va, \
#                 out_file_path = out_va, statistics_type='MEAN', \
#                 scale = 30)

# geemap.zonal_statistics(in_value_raster = collection_ma, in_zone_vector = wet_ma, \
#                 out_file_path = out_ma, statistics_type='MEAN', \
#                 scale = 30)

In [48]:
#Feesh